앞의 data를 (4,64)로 바꾸었다. 4가지 습관을 64칸의 시간표로 적어둔 것이다. 즉 특징이 4인 벡터이다. 이때 lstm의 encoder에 이 벡터를 넣어 학습시키고, latent를 통해 요약, decoder을 통해 다시 생성한다.

In [ ]:
import torch
import torch.nn as nn

class LSTMAutoEncoder(nn.Module):
    def __init__(
        self,
        n_mels=4,
        hidden_dim=128,
        latent_dim=64,
        num_layers=2
    ):
        super().__init__()

        # -------- Encoder --------
        self.encoder = nn.LSTM(
            input_size=n_mels,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )

        self.to_latent = nn.Linear(hidden_dim, latent_dim)

        # -------- Decoder --------
        self.from_latent = nn.Linear(latent_dim, hidden_dim)

        self.decoder = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=n_mels,
            num_layers=num_layers,
            batch_first=True
        )

    def forward(self, x):
        """
        x: (B, n_mels, T)
        return: reconstructed x (B, n_mels, T)
        """

        # (B, n_mels, T) → (B, T, n_mels)
        x = x.permute(0, 2, 1)

        # -------- Encoder --------
        enc_out, (h_n, _) = self.encoder(x)
        # h_n: (num_layers, B, hidden_dim)

        h_last = h_n[-1]                 # (B, hidden_dim)
        z = self.to_latent(h_last)       # (B, latent_dim)

        # -------- Decoder --------
        h_dec = self.from_latent(z)      # (B, hidden_dim)

        # repeat for each timestep
        T = x.size(1)
        h_dec_seq = h_dec.unsqueeze(1).repeat(1, T, 1)

        recon, _ = self.decoder(h_dec_seq)
        # recon: (B, T, n_mels)

        # (B, T, n_mels) → (B, n_mels, T)
        recon = recon.permute(0, 2, 1)

        return recon
